In [5]:
from pathlib import Path
import pandas as pd
import numpy as np

result = list(Path("../llms-ppl-preds/").rglob("*.tsv"))
result = [str(x) for x in result]
result

['../llms-ppl-preds/contrast-sets/IMDb/data/test.tsv',
 '../llms-ppl-preds/llama2-20231209/sentiment/combined/test.tsv',
 '../llms-ppl-preds/llama2-20231209/sentiment/new/test.tsv',
 '../llms-ppl-preds/llama2-20231209/sentiment/orig/test.tsv',
 '../llms-ppl-preds/llama2-20231209/NLI/revised_hypothesis/test.tsv',
 '../llms-ppl-preds/llama2-20231209/NLI/revised_combined/test.tsv',
 '../llms-ppl-preds/llama2-20231209/NLI/orig/test.tsv',
 '../llms-ppl-preds/llama2-20231209/NLI/all_combined/test.tsv',
 '../llms-ppl-preds/llama2-20231209/NLI/revised_premise/test.tsv',
 '../llms-ppl-preds/counterfactually-augmented-data/sentiment/orig/test.tsv',
 '../llms-ppl-preds/counterfactually-augmented-data/sentiment/new/test.tsv',
 '../llms-ppl-preds/counterfactually-augmented-data/sentiment/combined/test.tsv',
 '../llms-ppl-preds/counterfactually-augmented-data/NLI/all_combined/test.tsv',
 '../llms-ppl-preds/counterfactually-augmented-data/NLI/revised_premise/test.tsv',
 '../llms-ppl-preds/counterfact

In [6]:
result_dict = {
    'name' : [],
    'task' : [],
    'part' : [],
    'ppl' : [],
    'dist' : [],
    'fliprate' : [],

}   
for p in result: 
    df = pd.read_csv(p, sep='\t')

    

    assert ('ppl' in df.columns)
    part = p.split('/')[-2]
    task = p.split('/')[-3]
    name = p.split('/')[-4].split('-')[0]

    #if part != 'new' and 'revised' not in part and 'imdb' not in task.lower():
    #        continue

    if 'combined' in part:
        continue

    if part == 'new' or 'revised' in part or 'imdb' in task.lower():
  
        result_dict['name'].append(name)
        result_dict['task'].append(task)
        result_dict['part'].append(part)

        result_dict['ppl'].append(df['ppl'].mean().round(2))
        print(p)
        result_dict['dist'].append(df['dist'].mean().round(2))

        if task == 'sentiment' or task.lower() == 'imdb':
            result_dict['fliprate'].append(np.round(df.apply(lambda x: x['Sentiment'] == x['predicted_label'], axis=1).mean()*100, 2))
        elif task == 'NLI':
            result_dict['fliprate'].append(np.round(df.apply(lambda x: x['gold_label'] == x['predicted_label'], axis=1).mean()*100, 2))

results = pd.DataFrame.from_dict(result_dict)
results = results.sort_values(by=['task', 'part', 'name'])

../llms-ppl-preds/contrast-sets/IMDb/data/test.tsv
../llms-ppl-preds/llama2-20231209/sentiment/new/test.tsv
../llms-ppl-preds/llama2-20231209/NLI/revised_hypothesis/test.tsv
../llms-ppl-preds/llama2-20231209/NLI/revised_premise/test.tsv
../llms-ppl-preds/counterfactually-augmented-data/sentiment/new/test.tsv
../llms-ppl-preds/counterfactually-augmented-data/NLI/revised_premise/test.tsv
../llms-ppl-preds/counterfactually-augmented-data/NLI/revised_hypothesis/test.tsv
../llms-ppl-preds/mistral-20240118/NLI/revised_hypothesis/test.tsv
../llms-ppl-preds/mistral-20240118/NLI/revised_premise/test.tsv
../llms-ppl-preds/mistral-20240118/sentiment/new/test.tsv
../llms-ppl-preds/llama2_70b-20240318/sentiment/new/test.tsv
../llms-ppl-preds/llama2_70b-20240318/NLI/revised_premise/test.tsv
../llms-ppl-preds/llama2_70b-20240318/NLI/revised_hypothesis/test.tsv
../llms-ppl-preds/gpt4-20240318/sentiment/new/test.tsv
../llms-ppl-preds/gpt4-20240318/NLI/revised_hypothesis/test.tsv
../llms-ppl-preds/gpt4-

In [11]:
results

,name,task,part,ppl,dist,fliprate
0,contrast,IMDb,data,51.07,0.16,81.15
6,counterfactually,NLI,revised_hypothesis,65.67,0.19,79.75
20,gpt3.5,NLI,revised_hypothesis,51.30,0.19,41.50
14,gpt4,NLI,revised_hypothesis,58.35,0.21,65.88
2,llama2,NLI,revised_hypothesis,59.60,0.28,38.64
12,llama2_70b,NLI,revised_hypothesis,47.69,0.32,48.85
7,mistral,NLI,revised_hypothesis,39.06,0.46,44.11
17,mistral_56b,NLI,revised_hypothesis,62.77,0.33,37.79
5,counterfactually,NLI,revised_premise,74.89,0.17,59.13
19,gpt3.5,NLI,revised_premise,71.62,0.15,35.50


In [10]:
print(results.drop(columns=['task', 'part']).to_latex(index=False))

\begin{tabular}{lrrr}
\toprule
            name &   ppl &  dist &  fliprate \\
\midrule
        contrast & 51.07 &  0.16 &     81.15 \\
counterfactually & 65.67 &  0.19 &     79.75 \\
          gpt3.5 & 51.30 &  0.19 &     41.50 \\
            gpt4 & 58.35 &  0.21 &     65.88 \\
          llama2 & 59.60 &  0.28 &     38.64 \\
      llama2\_70b & 47.69 &  0.32 &     48.85 \\
         mistral & 39.06 &  0.46 &     44.11 \\
     mistral\_56b & 62.77 &  0.33 &     37.79 \\
counterfactually & 74.89 &  0.17 &     59.13 \\
          gpt3.5 & 71.62 &  0.15 &     35.50 \\
            gpt4 & 73.39 &  0.28 &     57.12 \\
          llama2 & 70.34 &  0.36 &     41.02 \\
      llama2\_70b & 70.74 &  0.33 &     58.20 \\
         mistral & 78.34 &  0.36 &     37.71 \\
     mistral\_56b & 75.57 &  0.34 &     24.32 \\
counterfactually & 48.03 &  0.14 &     85.66 \\
          gpt3.5 & 49.53 &  0.16 &     79.51 \\
            gpt4 & 49.05 &  0.29 &     94.03 \\
          llama2 & 46.99 &  0.64 &     78.26

/tmp/ipykernel_3745275/120168847.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(results.drop(columns=['task', 'part']).to_latex(index=False))
